In [ ]:
from pprint import pprint  # pretty-printer
from collections import defaultdict
import numpy as np
import pandas as pd

from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

import gensim
from pprint import pprint  # pretty-printer
from collections import defaultdict
import smart_open


In [ ]:
documents = []
nodes = []
with open('/kaggle/input/citation-prediction-altegrad/abstracts.txt', 'r', errors = 'replace') as f:
    for line in f:
        #do stuff
        node, abstract = line.split('|--|')
        documents.append(abstract)
        nodes.append(node)

# Universal Sentence Encoder Embedding

In [ ]:
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
def embed(input):
  return model(input)

In [ ]:
message_embeddings = []
for i,doc in enumerate(documents):
    if i % 10000 == 0:
        print(i)
    message_embeddings += [embed([doc]).numpy()]
    
    

In [ ]:
message_embeddings = np.concatenate(message_embeddings, axis = 0)
print(message_embeddings.shape)

In [ ]:
with open('USE_embedding.npy', 'wb') as f:
    np.save(f, message_embeddings)

# Doc2Vec embedding

In [ ]:

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus('/kaggle/input/citation-prediction-altegrad/abstracts.txt'))


model = gensim.models.doc2vec.Doc2Vec(vector_size=512, min_count=2, window = 7, epochs=30,
                                     sample = 1e-5, )
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save('docVec.model')


In [ ]:
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])


#docVec = gensim.models.doc2vec.Doc2Vec.load(PATH_models+'docVec.model')
vecs = model.dv.get_normed_vectors()
import numpy as np
with open('doc2vec_emb.npy', 'wb') as f:
    np.save(f, vecs)